<a href="https://colab.research.google.com/github/hatopopvr/MyHitbloqRank/blob/main/MyHitbloqRank_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MyHitbloqRank_En
<b>Info</b>
- Visualizes your ranking on [hitbloq](https://hitbloq.com/)'s various map pools.

<b>Data</b>
- Ranking Data from HitBloq API - [GitHub](https://github.com/DaFluffyPotato/hitbloq/blob/main/main.py)
- Ranked Map Count scraped from HitBloq Web Site - https://hitbloq.com/map_pool/{pool}
- PeakRank scraped from Hitbloq Web Site : https://hitbloq.com/user/{user_id}?pool={pool}


<b>Author</b>
- hatopop ([@hatopop_vr](https://twitter.com/hatopop_vr))

In [1]:
#@title # Input Information
#@markdown ---
#@markdown <h4>Input Data</h4>

#@markdown <font size="2">`player_id` : PlayerID for <b>ScoreSaber</b>. <b>Must</b> be changed. (and input HitBloq UserId 4 digits, first time only)<br/></font>

player_id =  76561198412839195#@param {type:"number"}

#markdown <font size="2">`google_drive_mount_path` ：GoogleDriveのマウント先です。変更不要。</font><br/>
google_drive_mount_path = "/content/drive" #param {type:"string"}

#@markdown <font size="2">`google_drive_dir_path` : The directory to save data in GoogleDrive. Optional change.</font>
google_drive_dir_path = "/MyDrive/MyBeatSaberAnalysis/data" #@param {type:"string"}

#@markdown <font size="2">`minimum_play_count` : Pools with fewer than `minimum_play_count` plays will not get data.</font>
minimum_play_count = 1 #@param {type:"integer"}


#markdown ---
#markdown <h4>Setting</h4>

#markdown <font size="2">`saved_player_score_is_enable`: Whether to use saved score data. Download only the difference.</font>
saved_player_score_is_enable = False #param {type:"boolean"}
#markdown <font size="2">`acc_recalq_override_is_enable`: Whether to use the recalculated Accuracy value based on the number of Notes and Combo.</font>
acc_recalq_override_is_enable = True #param {type:"boolean"}
#markdown <font size="2">`ranked_song_form_leaderboard_is_enable`: Ranked譜面クリア進捗用のレベル別譜面数をScoreSaberのLeaderBoardから取得し直すか。☑で使用。<br /></font>
ranked_song_from_leaderboard_is_enable = False
#markdown <font size="2">`ss_plus_is_enable`: AccRank区分にSS+を使用するか。☑で使用。  </font><br />
ss_plus_is_enable = True #param {type:"boolean"}
#markdown <font size="2">`ss_plus_val`:SS+の設定値です。必要に応じて変更ください。範囲は91-99。
ss_plus_val = 95 #param {type:"slider", min:91, max:99, step:1}
#markdown <font size="2">`latest`:The number of days to be treated as the most recent results. 0 is the same date as today in Japan time. If 1, from yesterday.
latest =  0#param {type:"integer"}

#@title Libraryの取得
# !pip install GitPython
!pip install pandas==1.5.0
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import tz
import time
from tqdm import tqdm
import requests
import math
# import git
import gc
from IPython.display import HTML, Javascript,Image
from google.colab import drive, files
import numpy as np
from PIL import Image
import copy
from bs4 import BeautifulSoup

#@title Timezone UTC->日本時間+9:00での実行日時の取得(tz_ja)
#@title Google Driveのマウント
drive.mount(google_drive_mount_path)

#@title その他内部設定値
# データ元のURL
## song_data_zip_git_url: ScoreSaberの全曲情報のzip(json)のURLです。変更なければそのままで。
song_data_zip_git_url = "https://github.com/andruzzzhka/BeatSaberScrappedData.git"
## ranked_excluded_data_git_url: ScoreSaberのRank譜面除外リスト(csv)のURLです。暫定的処置。
ranked_excluded_data_git_url = "https://github.com/hatopopvr/ScoreSaberRankedExcludedMaps.git"
## rankedmapdata_url: BeatSaverデータのcsvのURLです。らっきょさんデータ。
rankedmapdata_url = 'https://api.github.com/repos/rakkyo150/RankedMapData/releases'
# google drive内のdata置き場親フォルダ
data_path = r"{}{}".format(google_drive_mount_path, google_drive_dir_path)
# player情報の親フォルダ(data_pathの子フォルダ)
player_path = r"{}/players_data/{}".format(data_path, player_id)
## playerinfoの保存先
player_info_path = r"{}/player_info_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_path = r"{}/scores_full_{}.csv".format(player_path, player_id)
player_ranked_path = r"{}/scores_ranked_{}.csv".format(player_path, player_id)
## playerのscore関連保存先
player_score_pickle_path = r"{}/scores_full_{}.pkl".format(player_path, player_id)
player_ranked_pickle_path = r"{}/scores_ranked_{}.pkl".format(player_path, player_id)


# SS_plus設定値
ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)
# その他 colab表示など設定
pd.options.display.precision = 2
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
page_count=100 #ScoreSaberの1ページあたりのページ数

# 出力最大高さ
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)

#@title 列情報の設定
# Player Infoの記録用列 (TotalFC, RankedFCは別途結合)
cols_info =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"ReplayWatched"
    ,"ScoreDate"
    ,"TotalFC"
    ,"RankedFC"
    ,"TotalPlayRank"
    ,"TotalPlayJPRank"
    ,"RankedPlayRank"
    ,"RankedPlayJPRank"
    ,"TotalScoreRank"
    ,"TotalScoreJPRank"
    ,"RankedScoreRank"
    ,"RankedScoreJPRank"
    ,"AveRankedAccRank"
    ,"AveRankedAccJPRank"
]

# Player Infoの表示用列 (TotalFC, RankedFCは別途結合)
cols_info_sort =[
    "Pic"
    ,"name"
    ,"country"
    ,"pp"
    ,"rank"
    ,"countryRank"
    ,"role"
    ,"TotalScore"
    ,"RankedScore"
    ,"AveRankedAcc"
    ,"TotalPlay"
    ,"RankedPlay"
    ,"TotalFC"
    ,"RankedFC"
    ,"ReplayWatched"
    ,"ScoreDateJa"
]

# HitBloq用の列
cols_hitbloq = [
                #'history',
                'cover',
                'playlist_cover',
                'pool',
                'pool_shown_name',
                'ratio',
                'tier',
                'tier_img',
                'rank',
                'cr',
                'ranked_score_count',
                #'username',
                'ranked_map_count',
                'ratio',
                'tier', 'username',
                #'_id',
                # map detail----
                'priority',
                'player_count',
                # map detail----
                'owners',
                # map detail----
                'date_created',
                'views',
                'accumulation_constant',
                #'banner_title_hide',
                #'force_recalc',
                #'leaderboard_id_list',
                #'long_description',
                #'needs_cr_total_recalc',
                #'short_description',
                #'shown_name',
                #'third_party',
                #'cr_curve.type', 'cr_curve.points', 'cr_curve.baseline','cr_curve.cutoff', 'cr_curve.exponential'
                ]

cols_hitbloq_table = [
                'cover_img',
                'tier_img',
                'your_rank',
                'cr',
                'play_count',
                'popular'
                ]

#@title スタイルの設定

cover_image_size=70 #カバー,プロファイル画像のサイズ単位px
info_image_size=150 #Player画像のサイズ単位px

style_format = {
        "rank": "#{:,.0f}",
        "countryRank": "#{:,.0f}",
        "TotalScore": "{:,.0f}",
        "RankedScore": "{:,.0f}",
        "AveRankedAcc": "{:,.2f}%",
        "TotalPlay": "{:,.0f}",
        "TotalFC": "{:,.0f}",
        "RankedFC": "{:,.0f}",
        "ReplayWatched": "{:,.0f}",
        "Level": "{:,.0f}",
        "Stars": "{:,.2f}★",
        "Acc": "{:,.2f}%",
        "AccRecalq": "{:,.2f}%",
        "AccDiff": "{:,.2f}%",
        "MaxScore": "{:,.0f}",
        "MaxScoreRecalq": "{:,.0f}",
        "MaxScoreDiff": "{:,.0f}",
        "Score": "{:,.0f}",
        "Rank": "#{:,.0f}",
        "pp": "{:,.2f}pp",
        "PP": "{:,.2f}pp",
        "Miss": "{:,.0f}",
        "Bad": "{:,.0f}",
        "Combo": "{:,.0f}",
        "Score": "{:,.0f}",
        "Play": "{:,.0f}",
        "DailyPlay": "{:,.0f}",
        "Bpm": "{:,.0f}",
        "Duration": "{:,.0f}",
        "Notes": "{:,.0f}",
        "Nps": "{:,.2f}",
        "Njs": "{:,.0f}",
        "Bombs": "{:,.0f}",
        "Obstacles": "{:,.0f}",
        "NjsOffset": "{:,.2f}",
        "Upvotesratio": "{:,.2f}",
        "Upvotes": "{:,.0f}",
        "Downvotes": "{:,.0f}",
        "Days": "{:,.0f}",
        "Months": "{:,.0f}",
        "Song": "{:,.0f}",
        "RecentCleared": "{:,.0f}",
        "Cleared": "{:,.0f}",
        "NF": "{:,.0f}",
        "NotCleared": "{:,.0f}",
        "AlreadyCleared": "{:,.0f}",
        "NotClearedRate": "{:,.2f}%",
        "AlreadyClearedRate": "{:,.2f}%",
        "RecentClearedRate": "{:,.2f}%",
        "NFRate": "{:,.2f}%",
        "SSS": "{:,.0f}",
        ss_plus: "{:,.0f}",
        "SS": "{:,.0f}",
        "S": "{:,.0f}",
        "A": "{:,.0f}",
        "B": "{:,.0f}",
        "Other": "{:,.0f}",
        "SSS-Rate": "{:,.2f}%",
        "SS-Rate": "{:,.2f}%",
        ss_plus_rate: "{:,.2f}%",
        "S-Rate": "{:,.2f}%",
        "A-Rate": "{:,.2f}%",
        "B-Rate": "{:,.2f}%",
        "Other-Rate": "{:,.2f}%",
        "FC": "{:,.0f}",
        "RecentFC": "{:,.0f}",
        "AlreadyFC": "{:,.0f}",
        "NotFC": "{:,.0f}",
        "AlreadyFCRate": "{:,.2f}%",
        "RecentFCRate": "{:,.2f}%",
        "NotFCRate": "{:,.2f}%",
        "Song": "{:,.0f}",
        "TotalPlayRank": "#{:,.0f}",
        "TotalPlayJPRank": "#{:,.0f}",
        "RankedPlayRank": "#{:,.0f}",
        "RankedPlayJPRank": "#{:,.0f}",
        "TotalScoreRank": "#{:,.0f}",
        "TotalScoreJPRank": "#{:,.0f}",
        "RankedScoreRank": "#{:,.0f}",
        "RankedScoreJPRank": "#{:,.0f}",
        "AveRankedAccRank": "#{:,.0f}",
        "AveRankedAccJPRank": "#{:,.0f}",
     }

ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)

hitbloq_style_format = {
        "cr": "{:,.0f}cr",
        "rank": "#{:,.0f}",
        "ranked_score_count": "{:,.0f}",
        "ratio": "{:,.3f}"
     }


def color_negative_red(val):
    color = 'red' if val < 0 else 'blue'
    return 'color: %s' % color


# Player Info Table Format
styles_info = [
    dict(selector="td", props=[("font-size", "130%"),
                               ("text-align", "center"),
                               ("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ])
]

# Player Info Table Format
styles_data = [
    {"selector":"td", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
    {"selector":"th", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
    {"selector":"table", "props":[("background-color", "black"),
                               ("font-color", "white"),
                               ("font-weight", "bold"),
                               ]},
]


# タイムゾーンの設定-------------------------


#@title 各種フォルダの作成
# 各種パスのディレクトリ作成
## データ大元のフォルダ作成
if os.path.exists(data_path) == False:
    print('MyBeatSaberAnalytics用のデータ格納フォルダをGoogle Driveに新規作成します。')
    print('データ格納フォルダ:{}'.format(data_path))
    os.makedirs(data_path, exist_ok=True)
    print('作成が完了しました。')

## データ大元のフォルダ作成
if os.path.exists(player_path) == False:
    old_player_info_path = r"{}/player_info_{}.csv".format(data_path, player_id)
    print('PlayerID:{}用のデータ格納フォルダを新規作成します。'.format(player_id))
    print('playerフォルダ:{}'.format(player_path))
    print('作成が完了しました。')
    os.makedirs(player_path, exist_ok=True)
    if os.path.isfile(old_player_info_path):
        print('データ引継ぎのため、player_infoｂのcsvファイルを上記フォルダに移動させます。')
        print('移動前:{}'.format(old_player_info_path))
        print('移動後:{}'.format(player_info_path))
        shutil.move(old_player_info_path, player_info_path)
        print('移動が完了しました。')


#@title Get Player Info (df_info)
url = r"https://scoresaber.com/api/player/{}/full".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

#df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'"/>'
df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'" style="width:{}px;"/>'.format(info_image_size)
df_info["TotalScore"] = df_info["scoreStats.totalScore"]
df_info["RankedScore"] = df_info["scoreStats.totalRankedScore"]
df_info["AveRankedAcc"] = df_info["scoreStats.averageRankedAccuracy"]
df_info["TotalPlay"] = df_info["scoreStats.totalPlayCount"]
df_info["RankedPlay"] = df_info["scoreStats.rankedPlayCount"]
df_info["ReplayWatched"] = df_info["scoreStats.replaysWatched"]
df_info["ScoreDate"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

df_info["ScoreDateUtc"] = pd.to_datetime(df_info['ScoreDate'], utc=True)
_df_info_idx = df_info.set_index("ScoreDateUtc")
df_info["ScoreDateJa"] = _df_info_idx.index.tz_convert("Asia/Tokyo")

PlayCount = df_info["TotalPlay"][0]
RankedPlay = df_info["RankedPlay"][0]
RangeCount = math.ceil(PlayCount / page_count) + 1

#print("Player:{}, TotalPlayCount:{:,}, RankedPlayCount:{:,}, PageCount:{}".format(df_info["name"][0], PlayCount, RankedPlay, RangeCount))

# display(df_info[['Pic' ,'name','TotalPlay']].style.set_table_styles(styles_info).format(style_format, na_rep="-"))

# ----------------------------------
# HitBloq User IDの取得
# ----------------------------------
hitbloq_id_path = r"{}/HitBloqUserID.txt".format(player_path)
input_text = "Please Input HitBloq UserId 4 digits (first time only):"

if os.path.isfile(hitbloq_id_path):
    with open(hitbloq_id_path) as f:
        hitbloq_user_id = f.read()
        if len(hitbloq_user_id) == 4:
            print("HitBloq UserID:{}".format(hitbloq_user_id))
        else:
            hitbloq_id = input(input_text)
            with open(hitbloq_id_path, mode='w') as f:
                f.write(hitbloq_user_id)
            with open(hitbloq_id_path) as f:
                hitbloq_id = f.read()
                print("HitBloq UserID:{}".format(hitbloq_user_id))
else:
    hitbloq_user_id = input(input_text)
    with open(hitbloq_id_path, mode='w') as f:
        f.write(hitbloq_user_id)
    with open(hitbloq_id_path) as f:
        hitbloq_id = f.read()
        print("HitBloq UserID:{}".format(hitbloq_user_id))

# ----------------------------------
# HitBloq Player情報取得
# ----------------------------------
url = r"https://hitbloq.com/api/users/{}".format(hitbloq_user_id)
try:
    response = requests.get(url)
    res_data = response.json()
    df_hitbloq_info = json_normalize(res_data)
except:
    print('error')

df_hitbloq_info = json_normalize(res_data)

df_hitbloq_info["Pic"] = '<img src="' + df_hitbloq_info["profile_pic"]+'" style="width:{}px;"/>'.format(info_image_size)

display(df_hitbloq_info[['Pic' ,'username', '_id']].style.set_table_styles(styles_info).format(style_format, na_rep="-"))

# scoresaber ユーザーIDと照合
# if (df_info['name'][0]) == df_hitbloq_info['username'][0]:
if df_hitbloq_info['scoresaber_id'][0] == str(player_id):
    print('We have verified that the ID is correct. :)')
else:
    if os.path.isfile(hitbloq_id_path):
        os.remove(hitbloq_id_path)
    raise ValueError("ID is incorrect, please check ScoreSaberID and HitBloqID. :(")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.0 which is incompatible.
Mounted at /content/drive
HitBloq UserID:7572


,Pic,username,_id
0,,lolipop,7572


We have verified that the ID is correct. :)


# GetData from Hitbloq

In [2]:
#@title Get HitBloq MapPools
#@markdown Get a list of HitBloq map pools from API : https://hitbloq.com/api/map_pools <br >
#@markdown map_pools_all : List of HitBloq map pools<br >

def get_hitbloq_map_pools():
    """
    Get a list of HitBloq map pools using the HitBloq API.
    HitBloq API を使用して、HitBloq のマッププールのリストを取得します。

    Returns:
        list: A list of strings, each representing a map pool.
              文字列のリストで、各要素がマッププールを表します。

    Raises:
        Exception: If there's an error while fetching the map pools.
                   マッププールの取得中にエラーが発生した場合。
    """
    url = r"https://hitbloq.com/api/map_pools"
    try:
        response = requests.get(url)
        map_pools = response.json()
        print('map_pools_all:{} '.format(len(map_pools)))
        return map_pools
    except:
        print('error')
        raise Exception("Error fetching map pools")

map_pools_all = get_hitbloq_map_pools()
print(map_pools_all)


<IPython.core.display.Javascript object>

map_pools_all:89 
['comfy_jammers', 'gladde', 'maps_jaydz_likes', 'interesting', 'clearly_rankable', 'alliesfingerbang', 'jammers', 'bbbear', 'blarts', 'dariacore', 'accacalac', 'ghosted', 'accuracyjammers', 'cn_tech', 'pauls', 'crv2', 'actualacc', 'taddusjam', 'nuggetsaber', 'beatsage', 'v3', 'faster_midspeed', 'challenge_tech', 'challenge_mania', 'poodles', 'staryouh', 'modcharts', 'joshamops', 'kacki', 'specture7_map', 'modern_tech', 'creamsaber', 'best_map_pool', 'bsno', 'kojirase_tech', 'oodles', 'mm', 'funtech', 'best', 'nolanimationsfanclub', 'techtechtech', 'joesaber', 'bytesaber', 'challenge_acc', 'bangners', 'ura_tech', 'temch', 'codesofts_favorites', 'slider_tech', 'funmaps', 'bj', 'funny_map', 'aimedhades', 'purple_vibro', 'touhou', 'seasonal_saber', 'garshsaber', 'casual_poodles', 'monster_jammers', '90_degrees', 'bomber_reset', 'nailcliper', 'ass_growth', 'comfy_modcharts', 'scoresaber_legacy', 'fitbeat', 'speedtech', 'top50', 'funny_haha_pauls', 'tgsaber', 'cn_map', '4or

In [3]:
#@title Get HitBloq PlayerRanking
#@markdown API : https://hitbloq.com/api/player_rank/{pool}/{user_id} <br>
#@markdown df_player_ranks: Player's ranking data, CR, and play count for each map pool <br>
#@markdown map_pools: List of map pools the player is participating in <br>
#markdown df_player_ranks : プレイヤーのマッププール毎のプレイヤーランクデータ,CR,プレイカウント等 <br >
#markdown map_pools : プレイヤーの参加しているマッププールのリスト <br >


def get_player_rankings(user_id, map_pools, minimum_play_count=0):
    """
    Get the player rankings for the specified user in the provided map pools using the HitBloq API.
    HitBloq API を使用して、指定されたユーザーのマッププールのプレイヤーランキングを取得します。

    Args:
        user_id (str): The user ID for the player.
                       プレイヤーのユーザーID。
        map_pools (list): A list of map pool names to fetch the rankings.
                          ランキングを取得するマッププール名のリスト。
        minimum_play_count (int, optional): Minimum play count to filter the results. Default is 0.
                                            結果を絞り込むための最小プレイ回数。デフォルトは 0。

    Returns:
        pd.DataFrame: A DataFrame containing the player's rankings in the specified map pools.
                      指定されたマッププールのプレイヤーのランキングを含むデータフレーム。
    """
    url_template = r"https://hitbloq.com/api/player_rank/{}/{}"
    response = requests.get(url_template.format(map_pools[0], user_id))
    res_data = response.json()
    df_player_ranks = pd.json_normalize(res_data)

    for i in tqdm(range(1, len(map_pools))):
        url = url_template.format(map_pools[i], user_id)

        try:
            response = requests.get(url)
            res_data = response.json()
            if res_data['ranked_score_count'] >= minimum_play_count:
                df_player_ranks = df_player_ranks.append(pd.json_normalize(res_data), ignore_index=True)
        except:
            break

    return df_player_ranks

df_player_ranks = get_player_rankings(hitbloq_user_id, map_pools_all, minimum_play_count)

# print(df_player_ranks.columns)
print('map_pools: {} '.format(len(df_player_ranks['pool'].to_list())))

map_pools = df_player_ranks['pool'].to_list()
print(map_pools)

<IPython.core.display.Javascript object>

100%|██████████| 88/88 [00:20<00:00,  4.32it/s]

map_pools: 58 
['comfy_jammers', 'gladde', 'interesting', 'jammers', 'dariacore', 'accacalac', 'ghosted', 'accuracyjammers', 'pauls', 'crv2', 'actualacc', 'taddusjam', 'faster_midspeed', 'challenge_mania', 'poodles', 'modcharts', 'joshamops', 'creamsaber', 'best_map_pool', 'bsno', 'kojirase_tech', 'oodles', 'funtech', 'best', 'nolanimationsfanclub', 'techtechtech', 'joesaber', 'bytesaber', 'challenge_acc', 'ura_tech', 'temch', 'codesofts_favorites', 'funmaps', 'bj', 'funny_map', 'aimedhades', 'touhou', 'garshsaber', 'casual_poodles', 'monster_jammers', 'bomber_reset', 'ass_growth', 'comfy_modcharts', 'scoresaber_legacy', 'fitbeat', 'speedtech', 'funny_haha_pauls', '4orizonjammers', 'actual_music', 'bird', 'maldsaber', 'tech_map_power_hour', 'dd_paradise', 'exasfavorites', 'midspeed_acc', 'techhaven', 'vocaloid', 'furry']


In [4]:
#@title Get HitBloq RankedList
#@markdown RankedList from API: https://hitbloq.com/api/ranked_list/{pool} <br>
#@markdown df_ranked_list: Dataframe with detailed ranking data for each map pool <br>
#@markdown df_hitbloq: Dataframe with player's ranking data combined with the detailed data of map pools <br>
#markdown df_ranked_list : マッププール毎のランクの詳細データフレーム<br>
#markdown df_hitbloq : プレイヤーのランクデータにマッププールの詳細データを結合したデータフレーム<br>

def get_ranked_lists(map_pools):
    """
    Get the ranked lists for the specified map pools using the HitBloq API.
    HitBloq API を使用して、指定されたマッププールのランク付けされたリストを取得します。

    Args:
        map_pools (list): A list of map pool names to fetch the ranked lists.
                          ランク付けされたリストを取得するマッププール名のリスト。

    Returns:
        pd.DataFrame: A DataFrame containing the ranked lists for the specified map pools.
                      指定されたマッププールのランク付けされたリストを含むデータフレーム。
    """
    url_template = r"https://hitbloq.com/api/ranked_list/{}"
    response = requests.get(url_template.format(map_pools[0]))
    res_data = response.json()
    df_ranked_list = pd.json_normalize(res_data)

    for i in tqdm(range(1, len(map_pools))):
        url = url_template.format(map_pools[i])
        try:
            response = requests.get(url)
            res_data = response.json()
            df_ranked_list = df_ranked_list.append(pd.json_normalize(res_data), ignore_index=True)
        except:
            print('error')

    return df_ranked_list

df_ranked_list = get_ranked_lists(map_pools)
# print(df_ranked_list.columns)

df_hitbloq = pd.merge(df_player_ranks, df_ranked_list, left_on="pool", right_on="_id", how="left", suffixes=("", "_y"))
# print(df_hitbloq.columns)

<IPython.core.display.Javascript object>

100%|██████████| 57/57 [00:12<00:00,  4.71it/s]


In [5]:
#@title Get HitBloq RankedMap Count (Scraping)
#@markdown RankedMapCount scraped from Hitbloq Website: https://hitbloq.com/map_pool/{pool} <br>
#@markdown df_hitbloq['ranked_map_count']: Obtained number of ranked maps <br>
#markdown RankedMapCount scraped from Hitbloq Web Site : https://hitbloq.com/map_pool/{pool} <br >
#markdown df_hitbloq['ranked_map_count'] : 取得したランク付けされたマップ数 <br>
# -----------------------
# Get Ranked Map Count
# -----------------------

tqdm.pandas()

def func_hitbloq_rankedmap_count(_map_pool):
    """
    Get the ranked map count for the specified map pool by scraping the HitBloq website.
    HitBloq ウェブサイトをスクレイピングして、指定されたマッププールのランク付けされたマップ数を取得します。

    Args:
        _map_pool (str): The name of the map pool to fetch the ranked map count.
                        ランク付けされたマップ数を取得するマッププールの名前。

    Returns:
        int: The ranked map count for the specified map pool.
             指定されたマッププールのランク付けされたマップ数。

    """

    url = 'https://hitbloq.com/map_pool/{}'.format(_map_pool)
    try:
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        map_info = soup.find('div', class_='pool-info-container')
        map_info_lists = map_info.text.replace(' ', '').split('\n')

        for map_info_list in map_info_lists:
            if 'RankedMaps' in map_info_list:
                hitbloq_rankedmap_count = int(map_info_list.split(':')[1].replace(",",""))
                # print(_map_pool, hitbloq_rankedmap_count)
                return hitbloq_rankedmap_count
        return 0
    except:
        return 0

df_hitbloq['ranked_map_count'] = df_hitbloq['pool'].progress_apply(func_hitbloq_rankedmap_count)

<IPython.core.display.Javascript object>

100%|██████████| 58/58 [00:09<00:00,  6.15it/s]


In [6]:
#@title Get HitBloq PeakRank (Scraping)
#@markdown PeakRank scraped from Hitbloq Website: https://hitbloq.com/user/{user_id}?pool={pool} <br>
#@markdown df_hitbloq['peak_rank']: Obtained player's peak rank <br>
#markdown df_hitbloq['peak_rank'] : 取得したプレイヤーのピークランク。<br>

def get_peak_rank(user_id, map_pool):
    """
    Get the peak rank for the specified user and map pool by scraping the HitBloq website.
    HitBloq ウェブサイトをスクレイピングして、指定されたユーザーのマッププール毎のピークランクを取得します。

    Args:
        user_id (str): The user ID for the player.
                       プレイヤーのユーザーID。
        map_pool (str): The name of the map pool to fetch the peak rank.
                        ピークランクを取得するマッププールの名前。

    Returns:
        int: The peak rank for the specified user and map pool.
             指定されたユーザーとマッププールのピークランク。
    """
    url = 'https://hitbloq.com/user/{}?pool={}'.format(user_id, map_pool)
    try:
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        peak_rank = soup.find('span', class_='player-profile-pool-max-rank').text
        peak_rank = int(re.sub(r"[\r\n,# ]","", peak_rank).split(':')[1])
        return peak_rank
    except:
        return 0

df_hitbloq['peak_rank'] = df_hitbloq['pool'].progress_apply(lambda x: get_peak_rank(hitbloq_user_id, x))


<IPython.core.display.Javascript object>

100%|██████████| 58/58 [00:09<00:00,  6.04it/s]


# MyHitbloqRank

In [7]:
#@title MyHitbloqRank
sort_key = "ratio" #@param ["ratio","rank","cr","peak_rank", "peak_ratio", "priority", "views","ranked_score_count", "ranked_map_count"]
order = "asc" #@param ["asc", "desc"]
badge = "tier" #@param ["tier", "peak_tier", "tier&peak_tier"]
limit = 20 #@param {type:"slider", min:1, max:100, step:1}

order_by = True if order == "asc" else False

# Player Info Table Format
styles_data = [
    {"selector":"td", "props":[("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
    {"selector":"th", "props":[
                                ("padding-top", "0px"),
                               ("padding-bottom", "0px"),
                               ]},
    {"selector":"table", "props":[("background-color", "black"),
                               ("font-color", "white"),
                               ("font-weight", "bold")
                               ]}
]


def func_cover_img(x):
    if 'static' in x[0]:
        return """<a href='https://hitbloq.com/map_pool/{}' target='_blank' style='text-decoration:none;'>
                    <p style='text-align:center; font-size:1.8em; font-family:aller;'>{}</p>
                    </a>""".format(x[2], x[1])
    else:
        if x[3]:
            return """<a href='https://hitbloq.com/map_pool/{}' target='_blank'>
                    <img src='{}' style='width:22em;height:4.4em;margin:0px;padding:0px;'/>
                    </a>""".format(x[2],x[0])
        else:
            return """<div style='display:inline-block; position:relative; padding:0px 0px 0px 0px;margin:0px;'>
                        <a href='https://hitbloq.com/map_pool/{}' target='_blank'>
                            <img src='{}' style='width:22em;height:4.4em; margin:0px;padding:0px;'/>
                            <div style='position: absolute; left:50%; top:50%; text-align:center; transform: translate(-50%,-50%); font-size:1.8em; font-family:aller;'>{}</div>
                        </a>
                    </div>
                    """.format(x[2],x[0],x[1])


def func_tier_img(x):
    return """<a href='https://hitbloq.com/user/{}?pool={}' target='_blank'>
                <img src='https://hitbloq.com/static/ranks/{}.png' style='height:4.4em;margin:0px;padding:0px;'/>
            </a>""".format(hitbloq_user_id, x[1], x[0])

def func_rank_by_player(x):
    page_count = 50
    page = x[0] // page_count

    try:
        return """<a href='https://hitbloq.com/ladder/{}?page={:}' target='_blank'>
                #{:,}<br/>/{:,}<br/>({:,.1f}%)
                </a>""".format(x[2], page, x[0], x[1], x[0]/x[1]*100)
    except:
        return """<a href='https://hitbloq.com/ladder/{}?page={:}' target='_blank'>
                #{:,}<br/>/{:,}<br/>(---%)
                </a>""".format(x[2], page, x[0], x[1])

def func_play_count(x):
    return "{:,}<br/>/{:,}<br/>({:,.1f}%)".format(x[0],x[1],x[0]/x[1]*100)

def func_popular(x):
    return """{:,.0f}<br/>
            ----------<br/>
            {:,.0f}view""".format(x[0], x[1])

def func_cr(x):
    if sort_key == "peak_rank" or sort_key == "peak_ratio":
        return """Peak#{:,.0f}<br/>
                ----------<br/>
                {:,.0f}cr""".format(x[1], x[0])
    else:
        return """{:,.0f}cr<br/>
                ----------<br/>
                Peak#{:,.0f}""".format(x[0], x[1])

def func_ratio(x):
    return x[0]/x[1]

def func_tier(x):
    if x == []:
        return 'default/none'
    elif x < 0.001:
        return 'default/mystic'
    elif x < 0.01:
        return 'default/myth'
    elif x < 0.025:
        return 'default/master'
    elif x < 0.07:
        return 'default/diamond'
    elif x < 0.15:
        return 'default/platinum'
    elif x < 0.25:
        return 'default/gold'
    elif x < 0.5:
        return 'default/silver'
    elif x < 0.5:
        return 'default/bronze'

caption_styles = [dict(selector="caption",
                       props=[
                            # ("text-align", "center"),
                            ("text-align", "lett"),
                              ("font-size", "150%"),
                            #   ("font-weight", "bold"),
                            #   ("padding", "3px 0 3px 3px"),
                            #   ("border-left", "4px solid #D90E44"),
                              ("border-bottom", "1px solid"),
                            #   ("color", 'black'),
                            #   ("text-decoration", 'underline')
                              ]
                       )
                  ]

hitbloq_table_css = [
    # ヘッダの CSS
    {
        "selector": "th",
        "props": [
            ("background-color", "#282a36"),
            ("color", "white"),
            ("border-top", "1px solid #282a36"),
            ("text-align", "center"),
            ("padding-top", "0px"),
            ("padding-bottom", "0px"),
            ("font-family", "aller"),
        ],
    },
    # ボディの CSS
    {
        "selector": "td",
        "props": [
            ("background-color", "black"),
            ("color", "white"),
            ("border-top", "5px solid #282a36"),
            ("text-align", "center"),
            ("padding-top", "0px"),
            ("padding-bottom", "0px"),
            ("font-family", "aller"),
            ("font-size", "1.2em"),
        ],
    },
    # ボディの CSS
    {
        "selector": "a",
        "props": [
            ("background-color", "black"),
            ("color", "white"),
            ("font-family", "aller"),
        ],
    },
    # ボディの CSS
    {
        "selector": "caption",
        "props": [
            ("text-align", "lett"),
            ("font-size", "150%"),
            ("border-bottom", "1px solid"),
        ],
    },


]


cols_hitbloq_table_tier = [
                'cover_img',
                'tier_img',
                'your_rank',
                'peak&cr',
                'play_count',
                'popular'
                ]

cols_hitbloq_table_peak = [
                'cover_img',
                'peak_tier_img',
                'your_rank',
                'peak&cr',
                'play_count',
                'popular'
                ]

cols_hitbloq_table_tier_peak = [
                'cover_img',
                'tier_img',
                'peak_tier_img',
                'your_rank',
                'peak&cr',
                'play_count',
                'popular'
                ]

df_hitbloq['tier_img'] = df_hitbloq[['tier','pool']].apply(func_tier_img, axis=1)
df_hitbloq['cover_img'] = df_hitbloq[['cover','pool_shown_name','pool','banner_title_hide']].apply(func_cover_img, axis=1)
df_hitbloq['your_rank'] = df_hitbloq[['rank','player_count','pool','peak_rank']].apply(func_rank_by_player, axis=1)
df_hitbloq['play_count'] = df_hitbloq[['ranked_score_count','ranked_map_count']].apply(func_play_count, axis=1)
df_hitbloq['popular'] = df_hitbloq[['priority','views']].apply(func_popular, axis=1)

# peak
df_hitbloq['peak&cr'] = df_hitbloq[['cr','peak_rank']].apply(func_cr, axis=1)
df_hitbloq['peak_ratio'] = df_hitbloq[['peak_rank','player_count']].apply(func_ratio, axis=1)
df_hitbloq['peak_tier'] = df_hitbloq['peak_ratio'].apply(func_tier)
df_hitbloq['peak_tier_img'] = df_hitbloq[['peak_tier','pool']].apply(func_tier_img, axis=1)


if badge == "tier":
    cols_hitbloq_table_v1 = cols_hitbloq_table_tier
elif badge == "peak_tier":
    cols_hitbloq_table_v1 = cols_hitbloq_table_peak
elif badge == "tier&peak_tier":
    cols_hitbloq_table_v1 = cols_hitbloq_table_tier_peak
else:
    cols_hitbloq_table_v1 = cols_hitbloq_table

df_hitbloq[
            (1==1)
            & (df_hitbloq['cr'] > 0)
            & (df_hitbloq['ranked_score_count']>0)
        ].sort_values(sort_key, ascending=order_by).reset_index().drop(['history', 'username', 'tier', 'pool', 'index'], axis=1
        )[cols_hitbloq_table_v1][:limit].style.set_table_styles(hitbloq_table_css).format(
            hitbloq_style_format, na_rep="-"
        ).set_caption("MyHitbloqRank - {} - {}".format(datetime.now().strftime("%Y.%m.%d"), df_info["name"][0])
        )

        #.applymap(func_format_table)

<IPython.core.display.Javascript object>

,cover_img,tier_img,your_rank,peak&cr,play_count,popular
0,,,"#95/7,441(1.3%)","10,335cr ---------- Peak#0",67/344(19.5%),"185 ---------- 13,033view"
1,,,"#327/17,191(1.9%)","8,829cr ---------- Peak#0",64/620(10.3%),"81 ---------- 3,560view"
2,,,"#157/7,741(2.0%)","4,657cr ---------- Peak#0",51/475(10.7%),"89 ---------- 2,361view"
3,,,"#164/7,797(2.1%)","9,616cr ---------- Peak#0",46/416(11.1%),"103 ---------- 5,844view"
4,Funny map😂,,"#230/10,910(2.1%)","2,158cr ---------- Peak#0",10/42(23.8%),"74 ---------- 1,671view"
5,,,"#225/10,000(2.2%)","2,414cr ---------- Peak#0",24/0(inf%),"68 ---------- 4,093view"
6,,,"#186/7,198(2.6%)","6,623cr ---------- Peak#0",36/150(24.0%),"47 ---------- 2,447view"
7,Exa's Favorites,,"#303/10,690(2.8%)","2,864cr ---------- Peak#0",44/720(6.1%),"78 ---------- 3,211view"
8,creamsaber,,"#563/17,313(3.3%)","5,710cr ---------- Peak#0","122/2,249(5.4%)","10 ---------- 3,073view"
9,Interesting Maps,,"#555/15,506(3.6%)","10,239cr ---------- Peak#0",44/252(17.5%),"85 ---------- 7,011view"
